<a href="https://colab.research.google.com/github/Basifrank/NLP_with_GPT2/blob/master/conversationalAnalysis2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!git clone https://github.com/openai/gpt-2.git

fatal: destination path 'gpt-2' already exists and is not an empty directory.


In [0]:
import os
import json
import random
import re
import pandas as pd
from numpy import int64

In [0]:
os.chdir("gpt-2")

In [0]:
#!pip3 install tensorflow-gpu==1.12.0

In [7]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 604kB 8.3MB/s 
     |████████████████████████████████| 51kB 17.7MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=f6a40188c02a999717b4666e598482e56489e91d3765d93f40ea189554063ea5
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533170 sha256=70b87926e5cb1e20fce0a59925bf9627d5d0ac1a02bd835872f0fe470ec188c9
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [4]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 666kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 37.7Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 689kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:27, 52.0Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.45Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 38.7Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 37.6Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')

In [0]:
#!pip3 install tensorflow-gpu==1.12.0

In [0]:
import json
import os
import numpy as np
import tensorflow as tf
import model, sample, encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)

In [9]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> I went to a lounge to celebrate my birthday and
======================================== SAMPLE 1 ========================================
 my oldest my youngest in 2011," notes PR spokeswoman Julie Switzerland, to whom I watched. "At the end of the night we would walk along the promenade and it was cordial, if you like, to enjoy the evening. At one point I bought him my shirt to replace something he'd been wearing so quickly. He obviously couldn't tell. But we worked in close proximity, just hanging out while underlining emotions with ifr, the Countess."

Also: The Sting continues HBO's television battle: Another renewal?

PR at audition ground: Burgess interviews Margot Robbie, sp

KeyboardInterrupt: ignored